In [1]:
#import loadmat
from scipy.io import loadmat
import re
import matplotlib.pyplot as plt
import numpy as np
# import tqdm.notebook as tqdm
import torch
plot = True
use_gpu = False
recompute_classifiers = False
torch.manual_seed(6)
np.random.seed(6)

In [2]:
def func(x, a, b):
    return a * x + b
def substract_linear(x,y):
    a,b = curve_fit(func, x, y)[0]
    linear = func(x,a,b)
    # plt.plot(x,linear)
    sub = y-linear
    return sub

In [3]:
# load data

spikes = loadmat('data/peripheral/periphTextureData_scan_spikes.mat')
scanTime = loadmat('data/peripheral/periphTextureData_scanTime.mat')
speedList= loadmat('data/peripheral/periphTextureData_scan_speedList.mat')
textureNames = loadmat('data/peripheral/periphTextureData_scan_textureNames.mat')
afferenttypes = loadmat('data/peripheral/periphTextureData_scan_afferenttype.mat')

FileNotFoundError: [Errno 2] No such file or directory: 'data/peripheral/periphTextureData_scan_spikes.mat'

# Data Structure
There are 
#### Spikes
The spikes array is a 5D array of size speed, nerves, textures, trials, horizontal offset
In every item of the array there is an array of spike times
#### scanTime
The scanTime array is a 2D array of size nerves, 
#### speedList
The list of speed values used in the experiment
#### textureNames
The list of texture names used in the experiment
#### afferenttypes
The list of afferent types used in the experiment

In [ ]:
spikes = spikes['spikes_scan']
# scanTime = scanTime['scanTime_scan']
speedList = speedList['speedList_scan']
textureNames = textureNames['texture_scan']
afferenttypes = afferenttypes['afferenttype_scan']

## Spike dimension
Spikes = (speed,aff_type,texture,trial,horizontal)

In [ ]:
# DIMENSIONS
print(spikes.shape)
print(speedList.shape)
print(textureNames.shape)
print(afferenttypes.shape)

In [ ]:
print(speedList)

In [ ]:
# EXTRACTING AFFERENT TYPE
afferents = [re.findall(r'RA|SA|PC',str(afferent))[0] for afferent in afferenttypes[0]]
print(len(afferents))
print(afferents)

In [ ]:
# EXTRACTING TEXTURE NAMES
textureNames = [re.findall(r"\['(.*?)']",str(texture))[0] for texture in textureNames[0]]
print(len(textureNames))
print(textureNames)

In [ ]:
# PLOTTING RASTER PLOTS OF PC AFFERENTS
counter = 0
# progress_bar = tqdm.trange(spikes.shape[0]*spikes.shape[1]*spikes.shape[2]*spikes.shape[3])
selection = np.where(np.array(afferents) == 'PC')[0]
print(selection)
min_diff = 10
for speed in [0]:
    for nerve in selection:
        for texture in range(spikes.shape[2]):
            for trial in range(spikes.shape[3]):
                for horizontal in range(spikes.shape[4]):
                # print('nerve',nerve)
                # print(spikes[speed,nerve,texture,trial])
                    spikes_here = spikes[speed,nerve,texture,trial,horizontal]
                    plt.scatter(spikes_here,np.ones_like(spikes_here)*counter,s=0.1,color='k')
                    counter += 1
                    # progress_bar.update()
                    # print(spikes[speed,nerve,texture,trial].shape)
                    # print(spikes[speed,nerve,texture,trial].shape)
                    if (spikes_here.shape[1] > 0) & (spikes_here.shape[0] > 1):
                        min_diff_here = np.min(np.diff(spikes_here[:,0]))
                        if min_diff_here < min_diff:
                            min_diff = min_diff_here
                        # print(min_diff_here)
                    
# plt.xlim(0,0.5)
print(f'min_clk = {min_diff/1e-3} ms')


### Binning the spikes so that we can do an FFT over them
Selecting only the PC afferents and the textures that are stretch denim and silk jacquard

In [ ]:
def timestotrain(spiketimes,time,rate,pw):
    risetimes = spiketimes-pw/2
    falltimes = spiketimes+pw/2
    times = np.linspace(0,time,int(time*rate))
    volts = np.logical_or.reduce([np.logical_and(times>risetimes[i],times<falltimes[i])for i in range(len(spiketimes))])
    return times, volts
import matplotlib.pyplot as plt
import numpy as np
import tqdm.notebook as tqdm
counter = 0
# progress_bar = tqdm.trange(spikes.shape[0]*spikes.shape[1]*spikes.shape[2]*spikes.shape[3])
selection_afferents = np.where(np.array(afferents) == 'PC')[0]
texture_sel = ['Stretch Denim','Silk Jacquard','Flag/Banner','Hucktowel']
# texture_sel = textureNames
tmp = [np.array(textureNames) == texture_sel[i] for i in range(len(texture_sel))]
selection_texture = np.where(np.logical_or.reduce(tmp))[0]
# selection_texture = range(spikes.shape[2])
min_diff = 10
spikes_binned = []
labels = []
time = 1 #experiment duration
rate = 2e3 #binning rate
pw = 1/(2e3) #size of each spike (useful for nice looking FFT)
colors = plt.cm.get_cmap('Dark2')(np.linspace(0,1,len(np.unique(textureNames))))
for speed in [0]:
    for nerve in [23]:
        print(nerve)
        for texture in selection_texture:
            for trial in range(spikes.shape[3]):
                for horizontal in range(spikes.shape[4]):
                # print('texture',textureNames[texture])
                # print('nerve',nerve)
                # print(spikes[speed,nerve,texture,trial])
                    spikes_here = spikes[speed,nerve,texture,trial,horizontal]
                    if (spikes_here.shape[1] > 0) & (spikes_here.shape[0] > 1):
                        times,spike_clocked = timestotrain(spikes_here[:,0],time=time,rate=rate,pw=pw)
                        
                    # if (spikes[speed,nerve,texture,trial].shape[1] > 0) & (spikes[speed,nerve,texture,trial].shape[0] > 1):
                    #     times,spike_clocked = timestotrain(spikes[speed,nerve,texture,trial][:,0],time=time,rate=rate,pw=pw)
                    # # else:
                    #     times = np.linspace(0,2,int(2*2e3))
                    #     spike_clocked = np.zeros_like(times)
                    
                        spikes_binned.append(spike_clocked.astype(int))
                        events = np.where(spike_clocked)[0]
                        if plot:
                            plt.scatter(times[events],np.ones_like(times[events])*counter,s=0.1,color=colors[texture])
                        labels.append([speed,nerve,texture,trial])
                        # progress_bar.update()
                        counter += 1
spikes_binned = np.array(spikes_binned)
#create custom legend
texture_chosen = np.unique(selection_texture)
if plot:
    from matplotlib.lines import Line2D
    custom_lines = []
    custom_labels = []
    for text in texture_chosen:
        custom_lines.append(Line2D([0], [0], color=colors[text], lw=4))
        custom_labels.append(textureNames[text])
    plt.legend(custom_lines, custom_labels,bbox_to_anchor=(1,1),fancybox=True, shadow=True,title='Texture')
    print(np.where(spikes_binned))
# plt.legend()
labels = np.array(labels)

In [ ]:
# colors = plt.cm.get_cmap('Dark2')(np.linspace(0,1,len(np.unique(labels[:,2]))))
if plot:
    SMALL_SIZE = 10
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 14
    plt.rc('font', family='georgia', size=SMALL_SIZE)
    # plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE) 
    # print(labels[:,2])
    texture_num = len(np.unique(labels[:,2]))
    labels_ordered = np.argsort(labels[:,2])
    fig1,axis1 = plt.subplots(ncols = 2,nrows = texture_num,figsize=(10,10))
    fft_save = []
    label_fft = []
    for idx,i in enumerate(labels_ordered):
        # print(spikes_binned[:,i])
        # print(i)
        # print(np.where(spikes_binned[:,i]))
        myfft = np.fft.rfft(spikes_binned[i,:])
        # print(myfft)
        # print(labels[i,2])
        fft_save.append(np.abs(myfft)/np.max(np.abs(myfft)))
        label_idx = np.where(np.unique(labels[:,2]) == labels[i,2])[0][0]
        label_fft.append(label_idx)
        events = np.where(spikes_binned[i,:])[0]
        axis1[label_idx,0].scatter(times[events],np.ones_like(events)*idx*1.5,s=10,color='k',marker='|',linewidths=1)
        axis1[label_idx,1].plot(np.abs(myfft)/np.max(np.abs(myfft))+idx*0.2,color='k',alpha=0.5)
        axis1[label_idx,1].spines['top'].set_visible(False)
        axis1[label_idx,1].spines['right'].set_visible(False)
        axis1[label_idx,1].spines['left'].set_visible(False)
        axis1[label_idx,0].spines['top'].set_visible(False)
        axis1[label_idx,0].spines['right'].set_visible(False)
        axis1[label_idx,0].spines['left'].set_visible(False)
        # axis1[label_idx,0].set_yticks([0.8,2.8])
        axis1[label_idx,0].set_yticks([])
        axis1[label_idx,1].set_yticks([])
        if label_idx < texture_num-1:
            axis1[label_idx,0].set_xticks([])
        axis1[label_idx,1].set_xticks([])
        
        axis1[label_idx,1].set_xlim(25,600)
        axis1[label_idx,0].text(-0.2, 0.5, textureNames[labels[i,2]], horizontalalignment='center',verticalalignment='center', transform=axis1[label_idx,0].transAxes,fontsize=12)
    axis1[-1,0].set_xlabel('Time (s)')
    axis1[0,0].set_title('Time')
    axis1[0,1].set_title('Frequency')
    axis1[-1,1].set_xlabel('Frequency (Hz)')
    axis1[-1,1].set_xticks([100,200,300,400,500,600])
    axis1[-1,1].set_xticklabels([100,200,300,400,500,600])
    # axis1[-1,0].set_xticks([0,2e6])
    # axis1[-1,0].set_xticklabels([0,2e3/rate])
    fig1.tight_layout()
    #set white background
    fig1.patch.set_facecolor('white')
    fig1.savefig('peripheral_data.png',dpi=300)
        # axis1[1,0].set_xticks([0,2e6,4e6])
    # axis1[1,0].set_xticklabels([0,2e6/rate,4e6/rate])
    fft_save = np.stack(fft_save,axis=0)


In [ ]:
from models import *    
import tqdm.notebook as tqdm
import torch
device = 'cuda:0' if ((torch.cuda.is_available()) & use_gpu) else 'cpu'
parameters = {
    'C_Osc': 1,
    'v_Osc_threshold': 1,
    'tau_Osc_Ie': 0.01,
    'tau_Osc': 0.1,
    'reset_osc': 0,
    'I_minimum_osc': 100,
    'I_step_osc': 1,
    'refrac_Osc': 0.001,
    'TDE_to_Osc_current': 0,
    'C_TDE': 1,
    'v_TDE_threshold': 1,
    'tau_TDE': 0.01,
    'reset_TDE': 0,
    'tau_trg_TDE': 0.001,
    'tau_fac_TDE': 0.001,
    'gain_trg_TDE': 1,
    'gain_fac_TDE': 1,
    'refrac_TDE': 0.001,
    'neurons_n': 200,
    'clock_sim' : 1/rate,
    'device': device,
    'trials_per_stimulus': 1,
}
parameters['tmp_current_list'] = torch.exp(torch.linspace(parameters['I_minimum_osc'],parameters['I_minimum_osc'] + parameters['I_step_osc']*parameters['neurons_n'],parameters['neurons_n']))
sPLL_layer = sPLL(parameters)
subset = 50
spikes_binned_here = spikes_binned
labels_here = labels
dataset = torch.utils.data.TensorDataset(torch.from_numpy(spikes_binned_here).to_dense().float()[:,:1000], torch.from_numpy(labels_here[:,2]).long())
counter = 0
train_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)
osc_coll_nostim = []
time_progress = tqdm.trange(1000)
batch_progress = tqdm.trange(len(train_loader))
with torch.no_grad():
    for x, y in train_loader:
        parameters['trials_per_stimulus'] = x.shape[0]
        x = x.permute(1,0).to(parameters['device'])
        print(x.shape)
        sPLL_layer.initialize_state(parameters)
        spikes_osc_t = []
        if counter > 0:
            break
        counter += 1
        for t in range(x.shape[0]):
            data_in = x[t].unsqueeze(1).repeat(1,parameters['neurons_n'])
            tde_spikes, osc_spikes = sPLL_layer(data_in*0)
            spikes_osc_t.append(osc_spikes.to_sparse().to(torch.bool))
            # print(spikes_coll_t[-1])
            time_progress.update()

        osc_coll_nostim.append(torch.stack(spikes_osc_t))
osc_coll_nostim = torch.concatenate(osc_coll_nostim,dim=1).to_dense().cpu().numpy()
sum_osc_nostim = osc_coll_nostim.sum(axis=0)[0]


In [ ]:
rate_osc_nostim = (sum_osc_nostim*rate/1000).astype(int)
print(rate_osc_nostim)


In [ ]:
from models import *    
import tqdm.notebook as tqdm
import torch
device = 'cuda:0' if ((torch.cuda.is_available()) & use_gpu) else 'cpu'
parameters = {
    'C_Osc': 1,
    'v_Osc_threshold': 1,
    'tau_Osc_Ie': 0.01,
    'tau_Osc': 0.1,
    'reset_osc': 0,
    'I_minimum_osc': 100,
    'I_step_osc': 1,
    'refrac_Osc': 0.001,
    'TDE_to_Osc_current': 10,
    'C_TDE': 1,
    'v_TDE_threshold': 1,
    'tau_TDE': 0.01,
    'reset_TDE': 0,
    'tau_trg_TDE': 0.001,
    'tau_fac_TDE': 0.001,
    'gain_trg_TDE': 1,
    'gain_fac_TDE': 1,
    'refrac_TDE': 0.001,
    'neurons_n': 200,
    'clock_sim' : 1/rate,
    'device': device,
    'trials_per_stimulus': 1,
}
parameters['tmp_current_list'] = torch.exp(torch.linspace(parameters['I_minimum_osc'],parameters['I_minimum_osc'] + parameters['I_step_osc']*parameters['neurons_n'],parameters['neurons_n']))
sPLL_layer = sPLL(parameters)
spikes_binned_here = spikes_binned
labels_here = labels
dataset = torch.utils.data.TensorDataset(torch.from_numpy(spikes_binned_here)[:,:1000].to_dense().float(), torch.from_numpy(labels_here[:,2]).long())
counter = 0
train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False, num_workers=8)
tde_coll = []
osc_coll = []
in_coll = []
labels_coll = []
trace_ifac = []
trace_itrg = []
trace_isyn = []
time_progress = tqdm.trange(1000)
batch_progress = tqdm.trange(len(train_loader))
with torch.no_grad():
    for x, y in train_loader:
        parameters['trials_per_stimulus'] = x.shape[0]
        x = x.permute(1,0).to(parameters['device'])
        print(x.shape)
        sPLL_layer.initialize_state(parameters)
        spikes_tde_t = []
        spikes_osc_t = []
        trace_ifac_t = []
        trace_itrg_t = []
        trace_isyn_t = []
        in_coll_t = []
        time_progress.reset()
        if counter > 60:
            break
        counter += 1
        for t in range(x.shape[0]):
            data_in = x[t].unsqueeze(1).repeat(1,parameters['neurons_n'])
            tde_spikes, osc_spikes = sPLL_layer(data_in)
            spikes_tde_t.append(tde_spikes.to_sparse().to(torch.bool))
            trace_ifac_t.append(sPLL_layer.TDE.state.i_fac.clone())
            trace_itrg_t.append(sPLL_layer.TDE.state.i_trg.clone())
            trace_isyn_t.append(sPLL_layer.OSC.state.I.clone())
            spikes_osc_t.append(osc_spikes.to_sparse().to(torch.bool))
            in_coll_t.append(x[t].to_sparse().to(torch.bool))
            # print(spikes_coll_t[-1])
            time_progress.update()
    
        # spikes_coll.append(torch.stack(spikes_coll_t))
        labels_coll.append(y)
        tde_coll.append(torch.stack(spikes_tde_t))
        osc_coll.append(torch.stack(spikes_osc_t))
        trace_ifac.append(torch.stack(trace_ifac_t))
        trace_itrg.append(torch.stack(trace_itrg_t))
        trace_isyn.append(torch.stack(trace_isyn_t))
        in_coll.append(torch.stack(in_coll_t))
        batch_progress.update()


In [ ]:
tde_coll = torch.concatenate(tde_coll, dim=1)
osc_coll = torch.concatenate(osc_coll, dim=1)
trace_ifac = torch.concatenate(trace_ifac,dim=1)
trace_itrg = torch.concatenate(trace_itrg,dim=1)
trace_isyn = torch.concatenate(trace_isyn,dim=1)
in_coll = torch.concatenate(in_coll, dim=1)
labels_coll = torch.concatenate(labels_coll, dim=0)

In [ ]:
tde_coll = tde_coll.to_dense().cpu().numpy()
osc_coll = osc_coll.to_dense().cpu().numpy()
trace_ifac = trace_ifac.to_dense().cpu().numpy()
trace_itrg = trace_itrg.to_dense().cpu().numpy()
trace_isyn = trace_isyn.to_dense().cpu().numpy()
in_coll = in_coll.to_dense().cpu().numpy()
labels_coll = labels_coll.cpu().numpy()

In [ ]:
fig1,axis1 = plt.subplots(nrows=5,sharex=True,figsize=(10,10))
which_trial = 0
axis1[0].plot(trace_ifac[:,which_trial,:])
axis1[1].plot(trace_itrg[:,which_trial,:])
axis1[2].plot(trace_isyn[:,which_trial,:])
events = np.where(tde_coll[:,which_trial,:])
axis1[3].scatter(events[0],events[1],marker='.')
events = np.where(osc_coll[:,which_trial,:])
axis1[4].scatter(events[0],events[1],marker='.')
# axis1[0].set_xlim(0,200)
plt.figure()
print(tde_coll.shape)
tde_sum = tde_coll.sum(axis=0)
print(tde_sum)
tde_sum_r = tde_sum
plt.imshow(tde_sum_r,aspect='auto',interpolation='none')

In [ ]:
sorted = np.argsort(np.array(labels_coll))
tde_sum_r = tde_sum[sorted,:]
max_row = tde_sum_r.max(axis=1)
max_coll = tde_sum_r.max(axis=0)
tde_sum_r_norm = tde_sum_r/max_row[:,None]
print(np.array(labels_coll)[sorted])
plt.imshow(tde_sum_r_norm,aspect='auto',interpolation='None',cmap='viridis')
labels_name = np.array([textureNames[i] for i in np.array(labels_coll)[sorted]])
# plt.yticks(np.arange(0,61,3),labels_name[np.arange(0,61,3)])

## Classify texture
#### Input Spike Count

In [ ]:
from bio_decoding import decoding_input
from multiprocessing import Pool

if recompute_classifiers:
    epochs = 10000
    # with Pool(processes=8) as pool:
    #     last_accuracy_in = pool.starmap(decoding_input, [[in_coll,labels_coll,textureNames,device,epochs,i] for i in range(100)])
    last_accuracy_in= decoding_input(in_coll,labels_coll,textureNames,device,epochs_n=10000)
    torch.save(last_accuracy_in,'last_accuracy_in.pt')
else:
    last_accuracy_in = torch.load('last_accuracy_in.pt')
        ## print('Epoch {}, Loss {}, Accuracy {}'.format(epoch, np.array(loss_avg).mean(), np.array(accuracy_avg).mean()))

#### Input FFT

In [ ]:
from bio_decoding import decoding_fft

if recompute_classifiers:
    last_accuracy_fft= decoding_fft(fft_save, label_fft,textureNames,device,epochs_n=10000)
    torch.save(last_accuracy_fft,'last_accuracy_fft.pt')
else:
    last_accuracy_fft = torch.load('last_accuracy_fft.pt')

#### sPLL Spike Count

In [ ]:
from bio_decoding import decoding_spll
if recompute_classifiers:
    last_accuracy_spll= decoding_spll(tde_sum, labels_coll, textureNames, device,epochs_n=10000)
    torch.save(last_accuracy_spll,'last_accuracy_spll.pt')
else:
    last_accuracy_spll = torch.load('last_accuracy_spll.pt')

#### PEARSON CORRELATION

In [ ]:
print(in_coll.shape)
print(tde_sum.shape)
print(fft_save.shape)

In [ ]:
from scipy.stats.stats import pearsonr
tde_sum_r = np.stack([substract_linear(np.arange(0,tde_sum.shape[1]),tde_sum[i]) for i in range(tde_sum.shape[0])])
in_coll_sum = in_coll.sum(axis=0)
fft_max = np.argmax(fft_save[:,20:125],axis=1)+20
tde_min = np.argmin(tde_sum_r,axis=1)
tde_freq = np.array([sum_osc_nostim[i] for i in tde_min])
freqs = np.fft.rfftfreq(fft_save.shape[1],1/rate)
print(freqs)
print(fft_max)
fft_freq = np.array([freqs[i] for i in fft_max])
print(fft_freq)
pearson = np.corrcoef([in_coll_sum,tde_freq])
print(pearson)
pearson = np.corrcoef([tde_freq,fft_freq])
print(pearson)
print(rate)
# for trial in range(in_coll.shape[1]):
#     # print(pearsonr(in_coll[trial,:],tde_sum[trial,:]))
#     print(np.sum(in_coll[:,trial]))
#     print(sum_osc_nostim[np.argmin(tde_sum_r[trial,:])])
#     

In [ ]:
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14
from scipy.optimize import curve_fit

plt.rc('font', family='georgia', size=SMALL_SIZE)
# plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE) 
label_unique = np.unique(labels_coll)
texture_sel = ['Stretch Denim','Silk Jacquard','Flag/Banner','Hucktowel']
# texture_sel = textureNames
size_barplot = 0.1
total_time = in_coll[:,0].shape[-1]/rate
print(total_time)
tmp = [np.array(textureNames) == texture_sel[i] for i in range(len(texture_sel))]
selection_texture = np.where(np.logical_or.reduce(tmp))[0]
label_unique = selection_texture
textwidth = 6.50127 #inches
global_width = textwidth*2.5
global_height = global_width*0.9/3
# fig1,axis1 = plt.subplots(ncols = 3,nrows = len(label_unique),figsize=(global_width,global_height))
fig1 = plt.figure(figsize=(global_width,global_height))
fig2 = plt.figure(figsize=(global_width,global_height))
gs = fig1.add_gridspec(len(label_unique), 3)
axis1 = np.array([[fig1.add_subplot(gs[i,j]) for j in range(3)] for i in range(len(label_unique))])
for label_idx,label in enumerate(label_unique):
    sel = np.array(labels_coll) == label
    sel_w = np.where(sel)[0]
    axis1[label_idx,0].text(-0.2, 0.5, textureNames[label], horizontalalignment='center',verticalalignment='center', transform=axis1[label_idx,0].transAxes,fontsize=12,font='georgia')
    for i,idx in enumerate(sel_w):
        events = np.where(in_coll[:,idx])[0]
        myfft = np.fft.rfft(in_coll[:,idx])
        myfftfreqs = np.fft.rfftfreq(in_coll[:,idx].shape[-1],1/rate)
        # print(myfftfreqs)
        # print(times[events])
        times = np.arange(0,in_coll[:,idx].shape[-1])/rate
        # print(in_coll[:,idx].shape[-1])
        axis1[label_idx,0].scatter(times[events],np.ones_like(events)*(i*1.5+label_idx*1.51*len(sel_w)),s=10,color='k',marker='|',linewidths=1)
        axis1[label_idx,1].plot(myfftfreqs,np.abs(myfft)/np.max(np.abs(myfft))+i*0.08+label_idx*0.081*len(sel_w),color='k',alpha=0.5)
        axis1[label_idx,1].spines['top'].set_visible(False)
        axis1[label_idx,1].spines['right'].set_visible(False)
        axis1[label_idx,1].spines['left'].set_visible(False)
        axis1[label_idx,0].spines['top'].set_visible(False)
        axis1[label_idx,0].spines['right'].set_visible(False)
        axis1[label_idx,0].spines['left'].set_visible(False)
        axis1[label_idx,2].spines['top'].set_visible(False)
        axis1[label_idx,2].spines['right'].set_visible(False)
        axis1[label_idx,2].spines['left'].set_visible(False)
        axis1[label_idx,2].spines['bottom'].set_visible(False)
        
        # axis1[label_idx,0].set_yticks([0.8,2.8])
        axis1[label_idx,0].set_yticks([])
        axis1[label_idx,1].set_yticks([])
        if label_idx < texture_num-1:
            axis1[label_idx,0].set_xticks([])
        axis1[label_idx,1].set_xticks([])
        
        # axis1[label_idx,1].set_xlim(50,125)
    
    # tde_sum_r = tde_sum[sel,:]
    # sel_w = np.where(sel)[0]
    tde_w = tde_sum[sel,:]
    tde_sum_r = np.stack([substract_linear(np.arange(0,tde_w.shape[1]),tde_w[i]) for i in range(tde_w.shape[0])])
    # plt.figure()
    # plt.plot(tde_sum[sel_w].T)
    # plt.plot(tde_sum_r.T)
    max_row = tde_sum_r.max(axis=1)*1
    max_coll = tde_sum_r.max(axis=0)
    tde_sum_r_norm = tde_sum_r/tde_sum_r.max()#/max_coll[None,:]
    im = axis1[label_idx,2].imshow(tde_sum_r_norm,aspect='auto',interpolation='None',cmap='cividis')
    # axis1[label_idx,2].set_title(textureNames[label])
    axis1[label_idx,2].set_yticks([])
    # axis1[label_idx,2].set_ylabel('Trials')
    axis1[label_idx,1].set_xlim([88,250])
    
    # axis1[label_idx,2].set_xlim([25,125])
    axis1[label_idx,2].set_xticks([])
    axis1[label_idx,0].grid(False)
    axis1[label_idx,1].grid(False)
    axis1[label_idx,2].grid(False)
axis1[-1,0].set_xlabel('Time (s)')
axis1[0,0].set_title('Time')
axis1[0,1].set_title('Frequency')
axis1[0,2].set_title('sPLL Response')

axis1[-1,1].set_xlabel('Frequency (Hz)')
axis1[-1,1].set_xticks([100,150,200,250])
axis1[-1,2].set_xticks([0,50,100,150,199])
axis1[-1,2].spines['bottom'].set_visible(True)
axis1[-1,2].set_xticklabels(rate_osc_nostim[np.array([0,50,100,150,199])])
# axis1[-1,1].set_xticklabels([100,200,300,400,500,600])

axis1[-1,2].set_xlabel('CCO Frequency (Hz)')


cbar_ax = fig1.add_axes([1, 0.1, 0.01, 0.84])

colorbar = fig1.colorbar(im, cax=cbar_ax,orientation='vertical')
colorbar.set_label('Relative TDE Count')
colorbar.outline.set_visible(False)
# cbar_ax.set_yticks([0,1])
# cbar_ax.set_yticklabels([0,1])

## grid
letters = ['E','F','G']
gs = fig2.add_gridspec(len(label_unique), 3)
## add axes
cbar_ax2 = fig2.add_subplot(gs[:, 2])
# cbar_ax2 = fig1.add_axes([1.08, 0.1, size_barplot, 0.84])
xlocs = [0.33,0.66,0.99]
# print(last_accuracy_in)
# print(len(last_accuracy_fft))
accs = [last_accuracy_in,last_accuracy_fft,last_accuracy_spll]
accs_mean = [np.mean(acc)*100 for acc in accs]
accs_std = [np.std(acc)*100 for acc in accs]
# for acc in accs:
#     print(acc)
#     print(len(acc))
cbar_ax2.bar(xlocs[0],accs_mean[0],color='#7F7E7E',width=0.3,yerr=accs_std[0])
cbar_ax2.bar(xlocs[1],accs_mean[1],color='#32407B',width=0.3,yerr=accs_std[1])
cbar_ax2.bar(xlocs[2],accs_mean[2],color='#A13558',width=0.3,yerr=accs_std[2])

# cbar_ax2.set_xticks([0.25,0.75])
# cbar_ax2.set_xticklabels(['NULL','sPLL'])
who = ['Count','Frequency','sPLL']
for i, v in enumerate(accs_mean):
    cbar_ax2.text(xlocs[i] + 0, v+accs_std[i] + 2, str(int(v))+'% ±'+ str(int(accs_std[i])) + '%',verticalalignment='center',horizontalalignment='center')# cbar_ax2.set_ylabel('Accuracy (%)')
    cbar_ax2.text(xlocs[i] + 0, v+accs_std[i] + 6, who[i],verticalalignment='center',horizontalalignment='center',weight='bold')# cbar_ax2.set_ylabel('Accuracy (%)')
# i = 2
# v = accs[-1]
# cbar_ax2.text(xlocs[i] - 0.5, v + 2, str(v)+'%',verticalalignment='center',horizontalalignment='right')# cbar_ax2.set_ylabel('Accuracy (%)')
# cbar_ax2.text(xlocs[i] - 0.5, v + 5, who[i],verticalalignment='center',horizontalalignment='right',weight='bold')# cbar_ax2.set_ylabel('Accuracy (%)')
# cbar_ax2.legend(['Time','Frequency','sPLL'],loc='upper right')
# cbar_ax2.set_xlim([0.2,2])
cbar_ax2.set_yticks([])
cbar_ax2.set_xticks([])
cbar_ax2.spines['top'].set_visible(False)
cbar_ax2.spines['left'].set_visible(False)
cbar_ax2.spines['right'].set_visible(False)
cbar_ax2.set_ylim([0,120])
cbar_ax2.set_title('Accuracy')
letters = ['A','B','C','F']
for i in range(axis1.shape[1]):
    axis1[0,i].text(-0.1, 1.1, letters[i], horizontalalignment='left',verticalalignment='center', transform=axis1[0,i].transAxes,size=20, weight='bold')
cbar_ax2.text(-0.1, 1.1, letters[-1], horizontalalignment='center',verticalalignment='center', transform=cbar_ax2.transAxes,size=20, weight='bold')
fig1.tight_layout()
#set white background
fig1.patch.set_facecolor('white')
# cbar_ax = fig1.add_axes([0.85, 0.15, 0.05, 0.7])
# cbar_ax.set_xlabel('Normalized TDE Count')
# fig1.colorbar(im, cax=cbar_ax,orientation='vertical')
# print(global_width)
# print(fig1.get_figwidth())

fig1.savefig('peripheral_data_spll.png',dpi=300,bbox_inches='tight')
fig1.savefig('Fig2a.pdf',dpi=300,bbox_inches='tight')
# for label_idx,label in enumerate(label_unique):
# for label_idx,label in enumerate([0]):
#     sel = np.array(labels_coll) == label
from scipy.optimize import curve_fit




texture = 1
trial = 10
tde_sum_r = np.stack([substract_linear(np.arange(0,tde_sum.shape[1]),tde_sum[i]) for i in range(tde_sum.shape[0])])
sel = np.array(labels_coll) == texture
sel_w = np.where(sel)[0]
tde_sum_r_texture = np.stack([substract_linear(np.arange(0,tde_sum.shape[1]),tde_sum[i]) for i in sel_w])
which = sel_w[trial]
# tde_sum_r = [substract_linear(np.arange(0,tde_sum.shape[1]),tde_sum[i]) for i in range(tde_sum.shape[0])]

tde_sum_r_norm = (tde_sum_r-tde_sum_r.min())/(tde_sum_r.max()-tde_sum_r.min())
tde_sum_r_norm_texture = (tde_sum_r_texture-tde_sum_r_texture.min())/(tde_sum_r_texture.max()-tde_sum_r_texture.min())
tde_sum_r_norm_max = tde_sum_r_texture/tde_sum_r_texture.max()
# tde_sum_r_norm = (tde_sum_r/max_row[:,None])
myfft = np.fft.rfft(in_coll[:,which])
myfft_freqs = np.fft.rfftfreq(in_coll.shape[0],d=1/rate)
# fig1,axis1 = plt.subplots(len(label_unique),2,figsize=(20,8))
# fig2 = plt.figure(figsize=(global_width,global_height))
## grid
letters = ['E','D']
# gs = fig2.add_gridspec(4, 2)
## add axes
ax1 = fig2.add_subplot(gs[:, 1])
ax1.text(-0.1, 1.1, letters[0], horizontalalignment='center',verticalalignment='center', transform=ax1.transAxes,size=20, weight='bold')

ax1.plot(myfft_freqs,np.abs(myfft)/np.abs(myfft).max()*5 + 22,'k',linewidth=1)
min = np.argmin(np.diff(tde_sum_r_norm[which,:]))
colors = plt.cm.get_cmap('cividis')(np.linspace(0,1,10))

# print(textureNames[labels_coll[which]])
for osc in range(parameters['neurons_n']):
    # tde_value = (tde_sum_r_norm[which,osc] - tde_sum_r_norm[which,:].min())/(tde_sum_r_norm[which,:].max() - tde_sum_r_norm[which,:].min())
    tde_value = tde_sum_r_norm_texture[which,osc]
    # print(tde_value)
    color = colors[int(tde_value*9.99)]
    # print(color)
    myfft = np.abs(np.fft.rfft(osc_coll[:,which,osc]))
    myfftfreqs = np.fft.rfftfreq(osc_coll[:,which,osc].shape[0],1/rate)
    ax1.plot(myfftfreqs,myfft/myfft.max()+osc*0.08,color=color,linewidth=1)
    ax1.grid(False)
ax1.set_xlim([88,250])
ax1.set_xlabel('Frequency (Hz)')
ax1.set_title('FFT all CCOs vs Input - 1 Texture 1 Trial')
ax1.spines['left'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.set_yticks([])
ax1.legend(plt.plot([0,1],[0,1],'k'),[textureNames[labels_coll[which]]],loc='upper right',frameon=False,title='Input Texture',fontsize=10)
cbaxes = fig2.add_axes([0.4, 0.8, 0.07, 0.01])
cbaxes.set_title('Rel. TDE Count',fontsize=10)
# formatter = LogFormatter(10, labelOnlyBase=False)
import matplotlib.colors as colors_package
sm = plt.cm.ScalarMappable(cmap='cividis', norm=colors_package.Normalize(vmin=tde_sum_r_norm_max.min(), vmax=tde_sum_r_norm_max.max()))
fig2.colorbar(sm,cax=cbaxes, orientation='horizontal',cmap='Greys')
cbaxes.set_xticks([-0.6,0,1])
cbar_ax = fig2.add_axes([1, 0.1, 0.06, 0.84])
cbar_ax.axis('off')
# cbaxes.yaxis.set_ticks_position('left')
# cbaxes.yaxis.set_label_position('left')
# cbaxes.patch.set_facecolor('black')
# ax.patch.set_facecolor('black').set_facecolor((1.0, 0.47, 0.42))

# plt.figure()
# plt.plot(np.diff(tde_sum_r_norm[which,:]))
# plt.figure()
# fig0,axis0 = plt.subplots(len(label_unique))
# fig1,axis1 = plt.subplots(len(label_unique),2,figsize=(10,10))
# raise ValueError

ax3 = np.array([fig2.add_subplot(gs[i,0]) for i in range(len(label_unique))])

for label_idx,label in enumerate(label_unique):
    sel = np.where(np.array(labels_coll) == label)[0]
    # ax2 = fig2.add_subplot(gs[label_idx, 1])
    # ax3 = fig2.add_subplot(gs[label_idx, 0])
    ax3[label_idx].text(-0.2, 0.5,textureNames[label], horizontalalignment='center',verticalalignment='center', transform=ax3[label_idx].transAxes,fontsize=12,font='georgia')

    
    # print(sel)
    for s_idx,s in enumerate(sel):
        myfft = np.fft.rfft(in_coll[:,s])
        myfft_freqs = np.fft.rfftfreq(in_coll.shape[0],d=1/rate)
        min = np.argmin(tde_sum_r_norm[s,:])
        # axis0[label_idx].plot(np.diff(np.diff(np.diff(tde_sum_r_norm[s,:]))))
        # ax2.plot(np.abs(myfft)/np.abs(myfft).max()+s_idx*0.1,'k',linewidth=1)
        
        # tde_value = tde_sum_r_norm[s,min]
        # print(tde_value*9.9)
        ax3[label_idx].plot(myfft_freqs,np.abs(np.fft.rfft(osc_coll[:,s,min]))/np.abs(np.fft.rfft(osc_coll[:,s,min])).max()+s_idx*0.08+label_idx*0.081**len(sel),color='k',alpha=0.5)
    if label_idx == len(label_unique)-1:
        # ax2.set_xlabel('Frequency (Hz)')
        ax3[label_idx].set_xlabel('Frequency (Hz)')
            
    else:
        # ax2.set_xticks([])
        ax3[label_idx].set_xticks([])
    if label_idx == 0:
        # ax2.set_title('FFT Input')
        ax3[label_idx].set_title('FFT of winning CCO - All Textures All Trials')
        # ax2.text(-0.1, 1.1, letters[1], horizontalalignment='left',verticalalignment='center', transform=ax2.transAxes,size=20, weight='bold')
        # ax3[label_idx].text(-0.1, 1.2, letters[1], horizontalalignment='left',verticalalignment='center', transform=ax3[label_idx].transAxes,size=20, weight='bold')


    for ax in [ax3[label_idx]]:
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xlim(50,125)
        ax.set_yticks([])
    ax3[label_idx].set_xlim([88,250])
    ax3[label_idx].grid(False)
ax3[0].text(-0.1, 1.4, letters[1], horizontalalignment='left',verticalalignment='center', transform=ax3[0].transAxes,size=20, weight='bold')
fig2.tight_layout()
fig2.savefig('Fig2b.pdf',dpi=300,bbox_inches='tight')

### IMAGE FOR ABSTRACT

In [ ]:
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14
from scipy.optimize import curve_fit

plt.rc('font', family='georgia', size=SMALL_SIZE)
# plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE) 
label_unique = np.unique(labels_coll)
texture_sel = ['Stretch Denim','Silk Jacquard','Flag/Banner','Hucktowel']
# texture_sel = textureNames
size_barplot = 0.1
total_time = in_coll[:,0].shape[-1]/rate
print(total_time)
tmp = [np.array(textureNames) == texture_sel[i] for i in range(len(texture_sel))]
selection_texture = np.where(np.logical_or.reduce(tmp))[0]
label_unique = selection_texture
textwidth = 6.50127 #inches
global_width = textwidth*2.5
global_height = global_width*0.9/3
# fig1,axis1 = plt.subplots(ncols = 3,nrows = len(label_unique),figsize=(global_width,global_height))
fig1 = plt.figure(figsize=(global_width,global_height))
fig2 = plt.figure(figsize=(global_width,global_height))
gs = fig1.add_gridspec(len(label_unique), 3)
axis1 = np.array([[fig1.add_subplot(gs[i,j]) for j in range(3)] for i in range(len(label_unique))])
for label_idx,label in enumerate(label_unique):
    sel = np.array(labels_coll) == label
    sel_w = np.where(sel)[0]
    axis1[label_idx,0].text(-0.2, 0.5, textureNames[label], horizontalalignment='center',verticalalignment='center', transform=axis1[label_idx,0].transAxes,fontsize=12,font='georgia')
    for i,idx in enumerate(sel_w):
        events = np.where(in_coll[:,idx])[0]
        myfft = np.fft.rfft(in_coll[:,idx])
        myfftfreqs = np.fft.rfftfreq(in_coll[:,idx].shape[-1],1/rate)
        # print(myfftfreqs)
        # print(times[events])
        times = np.arange(0,in_coll[:,idx].shape[-1])/rate
        # print(in_coll[:,idx].shape[-1])
        axis1[label_idx,0].scatter(times[events],np.ones_like(events)*(i*1.5+label_idx*1.51*len(sel_w)),s=10,color='k',marker='|',linewidths=1)
        axis1[label_idx,1].plot(myfftfreqs,np.abs(myfft)/np.max(np.abs(myfft))+i*0.08+label_idx*0.081*len(sel_w),color='k',alpha=0.5)
        axis1[label_idx,1].spines['top'].set_visible(False)
        axis1[label_idx,1].spines['right'].set_visible(False)
        axis1[label_idx,1].spines['left'].set_visible(False)
        axis1[label_idx,0].spines['top'].set_visible(False)
        axis1[label_idx,0].spines['right'].set_visible(False)
        axis1[label_idx,0].spines['left'].set_visible(False)
        axis1[label_idx,2].spines['top'].set_visible(False)
        axis1[label_idx,2].spines['right'].set_visible(False)
        axis1[label_idx,2].spines['left'].set_visible(False)
        axis1[label_idx,2].spines['bottom'].set_visible(False)
        
        # axis1[label_idx,0].set_yticks([0.8,2.8])
        axis1[label_idx,0].set_yticks([])
        axis1[label_idx,1].set_yticks([])
        if label_idx < texture_num-1:
            axis1[label_idx,0].set_xticks([])
        axis1[label_idx,1].set_xticks([])
        
        # axis1[label_idx,1].set_xlim(50,125)
    
    # tde_sum_r = tde_sum[sel,:]
    # sel_w = np.where(sel)[0]
    tde_w = tde_sum[sel,:]
    tde_sum_r = np.stack([substract_linear(np.arange(0,tde_w.shape[1]),tde_w[i]) for i in range(tde_w.shape[0])])
    # plt.figure()
    # plt.plot(tde_sum[sel_w].T)
    # plt.plot(tde_sum_r.T)
    max_row = tde_sum_r.max(axis=1)*1
    max_coll = tde_sum_r.max(axis=0)
    tde_sum_r_norm = tde_sum_r/tde_sum_r.max()#/max_coll[None,:]
    im = axis1[label_idx,2].imshow(tde_sum_r_norm,aspect='auto',interpolation='None',cmap='cividis')
    # axis1[label_idx,2].set_title(textureNames[label])
    axis1[label_idx,2].set_yticks([])
    # axis1[label_idx,2].set_ylabel('Trials')
    axis1[label_idx,1].set_xlim([88,250])
    
    # axis1[label_idx,2].set_xlim([25,125])
    axis1[label_idx,2].set_xticks([])
    axis1[label_idx,0].grid(False)
    axis1[label_idx,1].grid(False)
    axis1[label_idx,2].grid(False)
axis1[-1,0].set_xlabel('Time (s)')
axis1[0,0].set_title('Time')
axis1[0,1].set_title('Frequency')
axis1[0,2].set_title('sPLL Response')

axis1[-1,1].set_xlabel('Frequency (Hz)')
axis1[-1,1].set_xticks([100,150,200,250])
axis1[-1,2].set_xticks([0,50,100,150,199])
axis1[-1,2].spines['bottom'].set_visible(True)
axis1[-1,2].set_xticklabels(rate_osc_nostim[np.array([0,50,100,150,199])])
# axis1[-1,1].set_xticklabels([100,200,300,400,500,600])

axis1[-1,2].set_xlabel('CCO Frequency (Hz)')


cbar_ax = fig1.add_axes([1, 0.1, 0.01, 0.84])

colorbar = fig1.colorbar(im, cax=cbar_ax,orientation='vertical')
colorbar.set_label('Relative TDE Count')
colorbar.outline.set_visible(False)
# cbar_ax.set_yticks([0,1])
# cbar_ax.set_yticklabels([0,1])

## grid
letters = ['E','F','G']
gs = fig2.add_gridspec(len(label_unique), 3)
## add axes
cbar_ax2 = fig2.add_subplot(gs[:, 2])
# cbar_ax2 = fig1.add_axes([1.08, 0.1, size_barplot, 0.84])
xlocs = [0.33,0.66,0.99]
# print(last_accuracy_in)
# print(len(last_accuracy_fft))
accs = [last_accuracy_in,last_accuracy_fft,last_accuracy_spll]
accs_mean = [np.mean(acc)*100 for acc in accs]
accs_std = [np.std(acc)*100 for acc in accs]
# for acc in accs:
#     print(acc)
#     print(len(acc))
cbar_ax2.bar(xlocs[0],accs_mean[0],color='#7F7E7E',width=0.3,yerr=accs_std[0])
cbar_ax2.bar(xlocs[1],accs_mean[1],color='#32407B',width=0.3,yerr=accs_std[1])
cbar_ax2.bar(xlocs[2],accs_mean[2],color='#A13558',width=0.3,yerr=accs_std[2])

# cbar_ax2.set_xticks([0.25,0.75])
# cbar_ax2.set_xticklabels(['NULL','sPLL'])
who = ['Count','Frequency','sPLL']
for i, v in enumerate(accs_mean):
    cbar_ax2.text(xlocs[i] + 0, v+accs_std[i] + 2, str(int(v))+'% ±'+ str(int(accs_std[i])) + '%',verticalalignment='center',horizontalalignment='center')# cbar_ax2.set_ylabel('Accuracy (%)')
    cbar_ax2.text(xlocs[i] + 0, v+accs_std[i] + 6, who[i],verticalalignment='center',horizontalalignment='center',weight='bold')# cbar_ax2.set_ylabel('Accuracy (%)')
# i = 2
# v = accs[-1]
# cbar_ax2.text(xlocs[i] - 0.5, v + 2, str(v)+'%',verticalalignment='center',horizontalalignment='right')# cbar_ax2.set_ylabel('Accuracy (%)')
# cbar_ax2.text(xlocs[i] - 0.5, v + 5, who[i],verticalalignment='center',horizontalalignment='right',weight='bold')# cbar_ax2.set_ylabel('Accuracy (%)')
# cbar_ax2.legend(['Time','Frequency','sPLL'],loc='upper right')
# cbar_ax2.set_xlim([0.2,2])
cbar_ax2.set_yticks([])
cbar_ax2.set_xticks([])
cbar_ax2.spines['top'].set_visible(False)
cbar_ax2.spines['left'].set_visible(False)
cbar_ax2.spines['right'].set_visible(False)
cbar_ax2.set_ylim([0,120])
cbar_ax2.set_title('Accuracy')
letters = ['C','D','E','H']
for i in range(axis1.shape[1]):
    axis1[0,i].text(-0.1, 1.1, letters[i], horizontalalignment='left',verticalalignment='center', transform=axis1[0,i].transAxes,size=20, weight='bold')
cbar_ax2.text(-0.1, 1.1, letters[-1], horizontalalignment='center',verticalalignment='center', transform=cbar_ax2.transAxes,size=20, weight='bold')
fig1.tight_layout()
#set white background
fig1.patch.set_facecolor('white')
# cbar_ax = fig1.add_axes([0.85, 0.15, 0.05, 0.7])
# cbar_ax.set_xlabel('Normalized TDE Count')
# fig1.colorbar(im, cax=cbar_ax,orientation='vertical')
# print(global_width)
# print(fig1.get_figwidth())

fig1.savefig('peripheral_data_spll.png',dpi=300,bbox_inches='tight')
fig1.savefig('4selina_2.pdf',dpi=300,bbox_inches='tight')
# for label_idx,label in enumerate(label_unique):
# for label_idx,label in enumerate([0]):
#     sel = np.array(labels_coll) == label
from scipy.optimize import curve_fit




texture = 1
trial = 10
tde_sum_r = np.stack([substract_linear(np.arange(0,tde_sum.shape[1]),tde_sum[i]) for i in range(tde_sum.shape[0])])
sel = np.array(labels_coll) == texture
sel_w = np.where(sel)[0]
tde_sum_r_texture = np.stack([substract_linear(np.arange(0,tde_sum.shape[1]),tde_sum[i]) for i in sel_w])
which = sel_w[trial]
# tde_sum_r = [substract_linear(np.arange(0,tde_sum.shape[1]),tde_sum[i]) for i in range(tde_sum.shape[0])]

tde_sum_r_norm = (tde_sum_r-tde_sum_r.min())/(tde_sum_r.max()-tde_sum_r.min())
tde_sum_r_norm_texture = (tde_sum_r_texture-tde_sum_r_texture.min())/(tde_sum_r_texture.max()-tde_sum_r_texture.min())
tde_sum_r_norm_max = tde_sum_r_texture/tde_sum_r_texture.max()
# tde_sum_r_norm = (tde_sum_r/max_row[:,None])
myfft = np.fft.rfft(in_coll[:,which])
myfft_freqs = np.fft.rfftfreq(in_coll.shape[0],d=1/rate)
# fig1,axis1 = plt.subplots(len(label_unique),2,figsize=(20,8))
# fig2 = plt.figure(figsize=(global_width,global_height))
## grid
letters = ['G','F']
# gs = fig2.add_gridspec(4, 2)
## add axes
ax1 = fig2.add_subplot(gs[:, 1])
ax1.text(-0.1, 1.1, letters[0], horizontalalignment='center',verticalalignment='center', transform=ax1.transAxes,size=20, weight='bold')

ax1.plot(myfft_freqs,np.abs(myfft)/np.abs(myfft).max()*5 + 22,'k',linewidth=1)
min = np.argmin(np.diff(tde_sum_r_norm[which,:]))
colors = plt.cm.get_cmap('cividis')(np.linspace(0,1,10))

# print(textureNames[labels_coll[which]])
for osc in range(parameters['neurons_n']):
    # tde_value = (tde_sum_r_norm[which,osc] - tde_sum_r_norm[which,:].min())/(tde_sum_r_norm[which,:].max() - tde_sum_r_norm[which,:].min())
    tde_value = tde_sum_r_norm_texture[which,osc]
    # print(tde_value)
    color = colors[int(tde_value*9.99)]
    # print(color)
    myfft = np.abs(np.fft.rfft(osc_coll[:,which,osc]))
    myfftfreqs = np.fft.rfftfreq(osc_coll[:,which,osc].shape[0],1/rate)
    ax1.plot(myfftfreqs,myfft/myfft.max()+osc*0.08,color=color,linewidth=1)
    ax1.grid(False)
ax1.set_xlim([88,250])
ax1.set_xlabel('Frequency (Hz)')
ax1.set_title('FFT all CCOs vs Input - 1 Texture 1 Trial')
ax1.spines['left'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.set_yticks([])
ax1.legend(plt.plot([0,1],[0,1],'k'),[textureNames[labels_coll[which]]],loc='upper right',frameon=False,title='Input Texture',fontsize=10)
cbaxes = fig2.add_axes([0.4, 0.8, 0.07, 0.01])
cbaxes.set_title('Rel. TDE Count',fontsize=10)
# formatter = LogFormatter(10, labelOnlyBase=False)
import matplotlib.colors as colors_package
sm = plt.cm.ScalarMappable(cmap='cividis', norm=colors_package.Normalize(vmin=tde_sum_r_norm_max.min(), vmax=tde_sum_r_norm_max.max()))
fig2.colorbar(sm,cax=cbaxes, orientation='horizontal',cmap='Greys')
cbaxes.set_xticks([-0.6,0,1])
cbar_ax = fig2.add_axes([1, 0.1, 0.06, 0.84])
cbar_ax.axis('off')
# cbaxes.yaxis.set_ticks_position('left')
# cbaxes.yaxis.set_label_position('left')
# cbaxes.patch.set_facecolor('black')
# ax.patch.set_facecolor('black').set_facecolor((1.0, 0.47, 0.42))

# plt.figure()
# plt.plot(np.diff(tde_sum_r_norm[which,:]))
# plt.figure()
# fig0,axis0 = plt.subplots(len(label_unique))
# fig1,axis1 = plt.subplots(len(label_unique),2,figsize=(10,10))
# raise ValueError

ax3 = np.array([fig2.add_subplot(gs[i,0]) for i in range(len(label_unique))])

for label_idx,label in enumerate(label_unique):
    sel = np.where(np.array(labels_coll) == label)[0]
    # ax2 = fig2.add_subplot(gs[label_idx, 1])
    # ax3 = fig2.add_subplot(gs[label_idx, 0])
    ax3[label_idx].text(-0.2, 0.5,textureNames[label], horizontalalignment='center',verticalalignment='center', transform=ax3[label_idx].transAxes,fontsize=12,font='georgia')

    
    # print(sel)
    for s_idx,s in enumerate(sel):
        myfft = np.fft.rfft(in_coll[:,s])
        myfft_freqs = np.fft.rfftfreq(in_coll.shape[0],d=1/rate)
        min = np.argmin(tde_sum_r_norm[s,:])
        # axis0[label_idx].plot(np.diff(np.diff(np.diff(tde_sum_r_norm[s,:]))))
        # ax2.plot(np.abs(myfft)/np.abs(myfft).max()+s_idx*0.1,'k',linewidth=1)
        
        # tde_value = tde_sum_r_norm[s,min]
        # print(tde_value*9.9)
        ax3[label_idx].plot(myfft_freqs,np.abs(np.fft.rfft(osc_coll[:,s,min]))/np.abs(np.fft.rfft(osc_coll[:,s,min])).max()+s_idx*0.08+label_idx*0.081**len(sel),color='k',alpha=0.5)
    if label_idx == len(label_unique)-1:
        # ax2.set_xlabel('Frequency (Hz)')
        ax3[label_idx].set_xlabel('Frequency (Hz)')
            
    else:
        # ax2.set_xticks([])
        ax3[label_idx].set_xticks([])
    if label_idx == 0:
        # ax2.set_title('FFT Input')
        ax3[label_idx].set_title('FFT of winning CCO - All Textures All Trials')
        # ax2.text(-0.1, 1.1, letters[1], horizontalalignment='left',verticalalignment='center', transform=ax2.transAxes,size=20, weight='bold')
        # ax3[label_idx].text(-0.1, 1.2, letters[1], horizontalalignment='left',verticalalignment='center', transform=ax3[label_idx].transAxes,size=20, weight='bold')


    for ax in [ax3[label_idx]]:
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xlim(50,125)
        ax.set_yticks([])
    ax3[label_idx].set_xlim([88,250])
    ax3[label_idx].grid(False)
ax3[0].text(-0.1, 1.4, letters[1], horizontalalignment='left',verticalalignment='center', transform=ax3[0].transAxes,size=20, weight='bold')
fig2.tight_layout()
fig2.savefig('4selina_3.pdf',dpi=300,bbox_inches='tight')

 ## FFT of Oscillators

In [ ]:
# for label_idx,label in enumerate(label_unique):
# for label_idx,label in enumerate([0]):
#     sel = np.array(labels_coll) == label
from scipy.optimize import curve_fit

texture = 1
trial = 10
tde_sum_r = np.stack([substract_linear(np.arange(0,tde_sum.shape[1]),tde_sum[i]) for i in range(tde_sum.shape[0])])
sel = np.array(labels_coll) == texture
sel_w = np.where(sel)[0]
tde_sum_r_texture = np.stack([substract_linear(np.arange(0,tde_sum.shape[1]),tde_sum[i]) for i in sel_w])
which = sel_w[trial]
# tde_sum_r = [substract_linear(np.arange(0,tde_sum.shape[1]),tde_sum[i]) for i in range(tde_sum.shape[0])]

tde_sum_r_norm = (tde_sum_r-tde_sum_r.min())/(tde_sum_r.max()-tde_sum_r.min())
tde_sum_r_norm_texture = (tde_sum_r_texture-tde_sum_r_texture.min())/(tde_sum_r_texture.max()-tde_sum_r_texture.min())
tde_sum_r_norm_max = tde_sum_r_texture/tde_sum_r_texture.max()
# tde_sum_r_norm = (tde_sum_r/max_row[:,None])
myfft = np.fft.rfft(in_coll[:,which])
# fig1,axis1 = plt.subplots(len(label_unique),2,figsize=(20,8))
fig2 = plt.figure(figsize=(global_width,global_height))
## grid
letters = ['E','F','G']
gs = fig2.add_gridspec(4, 2)
## add axes
ax1 = fig2.add_subplot(gs[:, 0])
ax1.text(-0.1, 1.01, letters[0], horizontalalignment='left',verticalalignment='center', transform=ax1.transAxes,size=20, weight='bold')

ax1.plot(np.abs(myfft)/np.abs(myfft).max()*5 + 22,'k',linewidth=1)
min = np.argmin(np.diff(tde_sum_r_norm[which,:]))
colors = plt.cm.get_cmap('cividis')(np.linspace(0,1,10))

# print(textureNames[labels_coll[which]])
for osc in range(parameters['neurons_n']):
    # tde_value = (tde_sum_r_norm[which,osc] - tde_sum_r_norm[which,:].min())/(tde_sum_r_norm[which,:].max() - tde_sum_r_norm[which,:].min())
    tde_value = tde_sum_r_norm_texture[which,osc]
    print(tde_value)
    color = colors[int(tde_value*9.99)]
    # print(color)
    myfft = np.abs(np.fft.rfft(osc_coll[:,which,osc]))
    ax1.plot(myfft/myfft.max()+osc*0.1,color=color,linewidth=1)
    ax1.grid(False)
ax1.set_xlim(50,125)
ax1.set_xlabel('Frequency (Hz)')
ax1.set_title('FFT all CCOs vs Input - 1 Texture 1 Trial')
ax1.spines['left'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.set_yticks([])
ax1.legend(plt.plot([0,1],[0,1],'k'),[textureNames[labels_coll[which]]],loc='upper right',frameon=False,title='Input Texture',fontsize=10)
cbaxes = fig2.add_axes([0.13, 0.82, 0.05, 0.01])
cbaxes.set_title('Rel. TDE Count',fontsize=10)
# formatter = LogFormatter(10, labelOnlyBase=False)
import matplotlib.colors as colors_package
sm = plt.cm.ScalarMappable(cmap='cividis', norm=colors_package.Normalize(vmin=tde_sum_r_norm_max.min(), vmax=tde_sum_r_norm_max.max()))
fig2.colorbar(sm,cax=cbaxes, orientation='horizontal',cmap='Greys')
cbaxes.yaxis.set_ticks_position('left')
cbaxes.yaxis.set_label_position('left')
cbaxes.patch.set_facecolor('black')
# ax.patch.set_facecolor('black').set_facecolor((1.0, 0.47, 0.42))

# plt.figure()
# plt.plot(np.diff(tde_sum_r_norm[which,:]))
# plt.figure()
# fig0,axis0 = plt.subplots(len(label_unique))
# fig1,axis1 = plt.subplots(len(label_unique),2,figsize=(10,10))
# raise ValueError
for label_idx,label in enumerate(label_unique):
    sel = np.where(np.array(labels_coll) == label)[0]
    # ax2 = fig2.add_subplot(gs[label_idx, 1])
    ax3 = fig2.add_subplot(gs[label_idx, 1])
    ax3.text(1.2, 0.5, textureNames[label], horizontalalignment='center',verticalalignment='center', transform=ax3.transAxes,fontsize=12,font='georgia')

    
    # print(sel)
    for s_idx,s in enumerate(sel):
        myfft = np.fft.rfft(in_coll[:,s])

        min = np.argmin(tde_sum_r_norm[s,:])
        # axis0[label_idx].plot(np.diff(np.diff(np.diff(tde_sum_r_norm[s,:]))))
        # ax2.plot(np.abs(myfft)/np.abs(myfft).max()+s_idx*0.1,'k',linewidth=1)
        
        # tde_value = tde_sum_r_norm[s,min]
        # print(tde_value*9.9)
        ax3.plot(np.fft.rfft(osc_coll[:,s,min])/np.fft.rfft(osc_coll[:,s,min]).max()+s_idx*0.1,color= 'k',linewidth=0.3)
    if label_idx == len(label_unique)-1:
        # ax2.set_xlabel('Frequency (Hz)')
        ax3.set_xlabel('Frequency (Hz)')
            
    else:
        # ax2.set_xticks([])
        ax3.set_xticks([])
    if label_idx == 0:
        # ax2.set_title('FFT Input')
        ax3.set_title('FFT of winning CCO - All Textures All Trials')
        # ax2.text(-0.1, 1.1, letters[1], horizontalalignment='left',verticalalignment='center', transform=ax2.transAxes,size=20, weight='bold')
        ax3.text(-0.2, 1.1, letters[1], horizontalalignment='left',verticalalignment='center', transform=ax3.transAxes,size=20, weight='bold')


    for ax in [ax3]:
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xlim(50,125)
        ax.set_yticks([])

fig2.savefig('Fig2b.pdf',dpi=300,bbox_inches='tight')
# 
# plt.figure()
# plt.plot(tde_sum[which,:],'k',linewidth=1)
# plt.figure()
# plt.plot(tde_sum_r[which,:],'k',linewidth=1)


In [ ]:
print(labels_coll)